In [1]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize
import matplotlib.lines as mlines
from scipy.interpolate import UnivariateSpline
from tqdm import tqdm
import pickle
import numpy as np

import crflux.models as pm

from MCEq.data import InteractionCrossSections
from MCEq.core import MCEqRun
import  mceq_underground_helper_forsplines as helper
from slope_param_functions import sl_depth

In [2]:
plot_dir = '/hetghome/khymon/Plots/cs-analysis/'


In [3]:
p1_values = [0.00, 0.05, -0.05, 0.00,  0.00, 0.00, 0.00]
p2_values = [0.00, 0.00,  0.00, 0.05, -0.05, 0.00, 0.00]
p3_values = [0.00, 0.00,  0.00, 0.00,  0.00, 0.05, -0.05]
season_values = ["jan", "apr", "jul"]  #  seasons



helper.initialize_flux_dicts(p1_values, p2_values, p3_values)
helper.some_function_that_uses_angles()
angles = helper.angles

Key: (0.0, 0.0, 0.0)
Length of cos_thetas: 3
Shape of flux for season jan: (3, 121)
Shape of flux for season apr: (3, 121)
Shape of flux for season jul: (3, 121)
Key: (0.05, 0.0, 0.0)
Length of cos_thetas: 3
Shape of flux for season jan: (3, 121)
Shape of flux for season apr: (3, 121)
Shape of flux for season jul: (3, 121)
Key: (-0.05, 0.0, 0.0)
Length of cos_thetas: 3
Shape of flux for season jan: (3, 121)
Shape of flux for season apr: (3, 121)
Shape of flux for season jul: (3, 121)
Key: (0.0, 0.05, 0.0)
Length of cos_thetas: 3
Shape of flux for season jan: (3, 121)
Shape of flux for season apr: (3, 121)
Shape of flux for season jul: (3, 121)
Key: (0.0, -0.05, 0.0)
Length of cos_thetas: 3
Shape of flux for season jan: (3, 121)
Shape of flux for season apr: (3, 121)
Shape of flux for season jul: (3, 121)
Key: (0.0, 0.0, 0.05)
Length of cos_thetas: 3
Shape of flux for season jan: (3, 121)
Shape of flux for season apr: (3, 121)
Shape of flux for season jul: (3, 121)
Key: (0.0, 0.0, -0.05

In [4]:
def derivative(param,angle):
    if param =='p1':
        delta_phi = (helper._flux(angle, "apr", p1=0.05, p2=0.0, p3=0.0, iecr=None) - helper._flux(angle, "apr", p1=-0.05, p2=0.0, p3= 0.0, iecr=None))
    elif param =='p2':
        delta_phi = helper._flux(angle, "apr", p1=0.0, p2=0.05, p3=0.0, iecr=None) - helper._flux(angle, "apr", p1=0.0, p2=-0.05, p3= 0.0, iecr=None)
    elif param =='p3':
        delta_phi = helper._flux(angle, "apr", p1=0.0, p2=0.0, p3=0.05, iecr=None) - helper._flux(angle, "apr", p1=0.0, p2=0.0, p3= -0.05, iecr=None)

    return delta_phi/(2*0.05)

In [5]:
helper.cos_thetas

array([1.       , 0.8660254, 0.5      ])

In [6]:
dphi_a = np.ndarray(shape=(len(angles),91))
dphi_b = np.ndarray(shape=(len(angles),91))
dphi_c = np.ndarray(shape=(len(angles),91))

for i in range(len(angles)):
  
    dphi_a[i] = derivative("p1",angles[i])
    dphi_b[i] = derivative("p2",angles[i])
    dphi_c[i] = derivative("p3",angles[i])

In [7]:
helper.cos_thetas

array([1.       , 0.8660254, 0.5      ])

In [8]:
# plot cs modification vs surface flux
flux_mod = helper._flux(0., "apr", p1=0.05, p2=0.0, p3=0.0, iecr=None) #+ helper._flux(0., "apr", p1=0.0, p2=0.05, p3=0.0, iecr=None) + helper._flux(0., "apr", p1=0.0, p2=0.0, p3=0.05, iecr=None)
flux_def = helper._flux(0., "apr", p1=0.0, p2=0.0, p3=0.0, iecr=None)

In [9]:
flux_mod/(flux_def)

array([0.99931995, 0.99934163, 0.99936537, 0.99938923, 0.99941433,
       0.99944132, 0.99946721, 0.99949124, 0.999512  , 0.99952888,
       0.99954121, 0.99954828, 0.99954955, 0.99954667, 0.99954952,
       0.99958897, 0.99973106, 1.00005831, 1.00056849, 1.00108174,
       1.00148297, 1.0017605 , 1.00218937, 1.00251075, 1.00279642,
       1.00296212, 1.00296169, 1.0027426 , 1.00226902, 1.00151562,
       1.00045048, 0.99905216, 0.9973156 , 0.99524489, 0.99288727,
       0.99038905, 0.98801774, 0.98601979, 0.98444359, 0.98329345,
       0.98256688, 0.98224528, 0.9823193 , 0.98277108, 0.98359155,
       0.98476226, 0.98628724, 0.9881208 , 0.99011684, 0.99205433,
       0.99381194, 0.99540067, 0.99681606, 0.99806177, 0.99913607,
       1.00003865, 1.00076933, 1.00132879, 1.00171795, 1.00193879,
       1.00199634, 1.00189652, 1.00166138, 1.00134821, 1.0010357 ,
       1.00076413, 1.00053405, 1.00034348, 1.00019167, 1.00007722,
       0.99999734, 0.99994827, 0.99992505, 0.99992219, 0.99993

In [10]:
flux_mod = (dphi_a * 0.05 + dphi_b * 0.05 + dphi_c * 0.05)+flux_def

flux_mod/flux_def

array([[0.99928131, 0.9993073 , 0.99933563, 0.99936393, 0.99939358,
        0.99942543, 0.99945594, 0.99948427, 0.99950888, 0.99952922,
        0.99954465, 0.99955458, 0.99955852, 0.99955803, 0.9995622 ,
        0.99959938, 0.99973041, 1.00003251, 1.00050635, 1.00098473,
        1.00135938, 1.00161763, 1.00201621, 1.00231756, 1.00258824,
        1.00275184, 1.00276688, 1.00258576, 1.00217573, 1.0015132 ,
        1.00056678, 0.99931279, 0.99774376, 0.99585879, 0.99369311,
        0.99135623, 0.98905223, 0.98697406, 0.98516983, 0.98365189,
        0.98242605, 0.98148128, 0.9808145 , 0.98041033, 0.98025985,
        0.98034179, 0.98065548, 0.98116756, 0.98179122, 0.98239461,
        0.98290369, 0.98333805, 0.98368747, 0.98396975, 0.9841904 ,
        0.98435578, 0.98446764, 0.984518  , 0.98452934, 0.98451203,
        0.98446566, 0.98440666, 0.98436449, 0.98441772, 0.9846776 ,
        0.98519298, 0.9859434 , 0.98692894, 0.98811646, 0.98946211,
        0.99092203, 0.99243571, 0.9939278 , 0.99

In [12]:
p1_values = [0.05]
p2_values = [0.05]
p3_values = [0.05]
season_values = ["jan", "apr", "jul"]  #  seasons



helper.initialize_flux_dicts(p1_values, p2_values, p3_values)

flux_mod = helper._flux(0., "apr", p1=0.05, p2=0.05, p3=0.05, iecr=None)
flux_mod/flux_def

Key: (0.05, 0.05, 0.05)
Length of cos_thetas: 3
Shape of flux for season jan: (3, 121)
Shape of flux for season apr: (3, 121)
Shape of flux for season jul: (3, 121)


array([0.99929851, 0.99932312, 0.99934997, 0.99937686, 0.99940507,
       0.99943537, 0.99946442, 0.9994914 , 0.99951482, 0.99953412,
       0.99954865, 0.99955779, 0.99956104, 0.99955994, 0.99956359,
       0.99960037, 0.9997312 , 1.00003317, 1.00050629, 1.00098372,
       1.00135779, 1.0016163 , 1.00201531, 1.00231684, 1.00258717,
       1.00274936, 1.00276146, 1.00257569, 1.00215971, 1.001491  ,
       1.00054022, 0.99928685, 0.99772734, 0.99586577, 0.99374256,
       0.99146949, 0.98924684, 0.9872591 , 0.98554745, 0.98411842,
       0.98297275, 0.98209565, 0.98148103, 0.98111194, 0.98097864,
       0.98106035, 0.98135693, 0.98183826, 0.98242479, 0.98299403,
       0.98347616, 0.98388918, 0.9842225 , 0.98449273, 0.98470483,
       0.98486469, 0.98497392, 0.98502491, 0.98503964, 0.98502853,
       0.98499165, 0.98494539, 0.98491802, 0.9849831 , 0.98524386,
       0.98574483, 0.98646622, 0.98740874, 0.98854173, 0.98982484,
       0.99121808, 0.99266528, 0.99409559, 0.99545979, 0.99669

In [9]:
# plot dN/dNmu
d = 1.5
angle = 0.
X = sl_depth(d,angle)


threshold = 500
p1 = 0.05
p2 = 0.
p3 = 0.
season = "apr"

1e4*helper.mult_dist(X, angle,
                        pm.GlobalSplineFitBeta(),"yields_" + season, 
                        p1, p2,p3,
                        norm=False,threshold=threshold#))[0]
                    ) / helper.rates(X, angle, season, p1, p2, p3, threshold)

array([1.99656776e-03, 1.81026468e-04, 4.25520437e-05, 1.46113949e-05,
       6.22316282e-06, 3.03160918e-06, 1.63016977e-06, 9.48314122e-07,
       5.88972913e-07, 3.83016549e-07, 2.58651224e-07, 1.82732345e-07,
       1.33024917e-07, 9.90184977e-08, 7.56784924e-08, 5.87088728e-08,
       4.65523583e-08, 3.74632395e-08, 3.05456400e-08, 2.51093577e-08,
       2.09375036e-08, 1.76198383e-08, 1.49508423e-08, 1.27383282e-08,
       1.09687462e-08, 9.50330688e-09, 8.28015244e-09, 7.25182939e-09,
       6.34121833e-09, 5.60878714e-09, 4.98111442e-09, 4.44040925e-09,
       3.97239946e-09, 3.56550682e-09, 3.21188799e-09, 2.90023885e-09,
       2.62608390e-09, 2.38406803e-09, 2.16971649e-09, 1.97927210e-09,
       1.80956563e-09, 1.64844321e-09, 1.51398329e-09, 1.39314628e-09,
       1.28428868e-09, 1.18599650e-09, 1.09705174e-09, 1.01640039e-09,
       9.43125044e-10, 8.76425115e-10, 8.11643264e-10, 7.56577589e-10,
       7.06174008e-10, 6.59962995e-10, 6.17529554e-10, 5.78506071e-10,
      

In [10]:
def dNdN_mu(d, p1, p2, p3):
    
    angle = 0.
    X = sl_depth(d,angle)
    season = "apr"

    return 1e4*helper.mult_dist(X, angle,
                        pm.GlobalSplineFitBeta(),"yields_" + season, 
                        p1, p2,p3,
                        norm=False,threshold=threshold#))[0]
                    ) / helper.rates(X, angle, season, p1, p2, p3, threshold)

In [ ]:
# plot in log log for 1.5 km depth and 2.5k